# Top loss analysis

> Analyze the top loss, from BERT classification predicition, against manual rule base label

In [289]:
import pandas as pd
import numpy as np
import os
from path import Path

In [290]:
HOME = Path(os.environ["HOME"])
DATA = HOME/"data"

LOSS_FILE = DATA/"cntext_loss2.csv"

In [291]:
df = pd.read_csv(LOSS_FILE)

In [292]:
df.sample(10)

,id,url,title,preview,isy,loss,loss2
59746,131153,https://zh.wikisource.org/wiki?curid=131153,對貌似溫敏判,對貌似溫敏判\n\n<onlyinclude>（儒生溫敏歿，或有貌似敏者，而弟子共師之。縣令...,True,0.001660,1.520550e-03
2452,189007,https://zh.wikisource.org/wiki?curid=189007,民國演義/6,民國演義/6\n\n 卻說清太保世續，召袁總理世凱入宮，當由隆裕后問及優待條件，曾否寄往南...,False,0.409088,6.502518e-01
197728,328518,https://zh.wikisource.org/wiki?curid=328518,中华人民共和国国务院令第500号,中华人民共和国国务院令第500号\n\n<section begin=国务院令 /><sec...,False,0.002379,3.039791e-05
72482,606738,https://zh.wikisource.org/wiki?curid=606738,與友人論服制書,與友人論服制書\n\n<onlyinclude>\n增三年之喪為三十六月，起於唐弘文館直學士...,True,0.000128,9.334498e-04
295592,281454,https://zh.wikisource.org/wiki?curid=281454,文苑英華 (四庫全書本)/卷0415,文苑英華 (四庫全書本)/卷0415\n\n<onlyinclude><poem> 欽定四...,True,-0.000000,1.192093e-07
12378,341528,https://zh.wikisource.org/wiki?curid=341528,海國春秋/35,海國春秋/35\n\n<onlyinclude>\n 話說雙龍之北最大的島曰屏風島。其四面...,True,0.000939,4.825535e-02
74354,119625,https://zh.wikisource.org/wiki?curid=119625,縣令箴,縣令箴\n\n<onlyinclude>諮爾多士，各司厥官。政不欲猛，刑不欲寬。寬則人慢，猛...,True,0.000267,8.715169e-04
310753,308828,https://zh.wikisource.org/wiki?curid=308828,道園學古録 (四庫全書本)/卷33,道園學古録 (四庫全書本)/卷33\n\n<onlyinclude><poem> 欽定四庫...,True,-0.000000,0.000000e+00
220130,187340,https://zh.wikisource.org/wiki?curid=187340,中华人民共和国国务院公报/2000年/第10号,中华人民共和国国务院公报/2000年/第10号\n,False,0.056589,1.656995e-05
59131,20434,https://zh.wikisource.org/wiki?curid=20434,周書/卷01,周書/卷01\n\n太祖文皇帝姓宇文氏，諱泰，字黑獺，代武川人也。其先出自炎帝神農氏，為黃帝...,True,0.000012,1.559829e-03


In [293]:
from ipywidgets import interact,interact_manual
from IPython.display import display

In [294]:
current_list = []

In [295]:
def tag_false(r_list = []):
    global current_list
    global df
    for i in r_list:
        current_list.remove(i)
    df.loc[df.id.isin(r_list),"loss2"] = df.loc[df.id.isin(r_list),"loss2"].apply(lambda x:-abs(x))
    df.loc[df.id.isin(current_list),"isy"] = False

In [299]:
def passneg(row):
    if row["loss"]<0:
        return -abs(row["loss2"])
    else:
        return row["loss2"]

df.loss2 = df.apply(passneg,axis=1)

In [296]:
@interact
def preview_top(page=(1,20,1)):
    global current_list 
    stat = pd.DataFrame(df.isy.value_counts())
    stat = stat.join(pd.DataFrame(df.loss.apply(lambda x:True if x<0 else False).value_counts()))
    stat = stat.join(pd.DataFrame(df.loss2.apply(lambda x:True if x<0 else False).value_counts()))
    display(stat)
    
    subdf = df[df.isy].sort_values(by="loss2",ascending=False).head(page*20).tail(20)
    current_list = list(subdf.id)
    return subdf

interactive(children=(IntSlider(value=10, description='page', max=20, min=1), Output()), _dom_classes=('widget…

In [355]:
tag_false([])

### Set true

In [301]:
@interact_manual
def set_true(kw ="kw",col = ["title","preview"]):
    if kw == "kw": return None
    subdf = df[df[col].str.contains(kw)]
    df.loc[df[col].str.contains(kw),"isy"] = True
    df.loc[df[col].str.contains(kw),"loss2"] = subdf.loss.apply(lambda x: - abs(x))
    return subdf

interactive(children=(Text(value='kw', description='kw'), Dropdown(description='col', options=('title', 'previ…

### Set false

In [300]:
@interact_manual
def set_false(kw ="kw",col = ["title","preview"]):
    if kw == "kw": return None
    subdf = df[df[col].str.contains(kw)]
    df.loc[df[col].str.contains(kw),"isy"] = False
    df.loc[df[col].str.contains(kw),"loss2"] = subdf.loss.apply(lambda x: - abs(x))
    return subdf

interactive(children=(Text(value='kw', description='kw'), Dropdown(description='col', options=('title', 'previ…

In [358]:
list(df[df["preview"].str.contains('的策略')].title)

['目前時局與我們的策略',
 '从独裁到民主/第三章',
 '红四方面军政治部: 筹款须知',
 '黨的策略應當是發展遊擊戰爭',
 '國民黨的一個根本問題',
 '我们在现阶段政治斗争的策略问题',
 '黨的中心工作要集中在倒蔣',
 '在中共“八七”會議上的發言',
 '目前中國革命的中心問題',
 '說呼全傳/023']

In [359]:
df.to_csv(DATA/"save_cache_cntext.csv",index = False)

In [360]:
df.to_csv(DATA/"cntext_v3.csv",index = False)